In [ ]:
import pandas as pd
import numpy as np
from os import walk

In [ ]:
names = ['squareId','timeInterval','countryCode','smsIn','smsOut','callIn','callOut','Internet']

In [ ]:
def readDir(dir_,names):
    #used to read all the files in a dir and return list of panda DataFrames
    #input dirpath and headers for dataFrames
    #output list of dataFrames
    result = []
    for _,_,file in walk(dir_):
        for f in file:
            data = pd.read_table(dir_+'/'+f,names = names)
            result.append(data)
    print('Reading Finished. There are '+str(len(result))+' files in all.')
    return result

In [6]:
Nov = []
dir_ = 'D:/MobileTrafficPrediction/data/raw/Nov'
for _,_,file in walk(dir_):
    for f in file:
        data = pd.read_table(dir_+'/'+f,names = names)
        Nov.append(data)
    print('Reading Finished. There are '+str(len(Nov))+' files in all.')

Reading Finished. There are 30 files in all.


In [7]:
Dec = []
dir_ = '../data/raw/milan/sms-call-internet-mi/sms-call-internet-mi-Dec'
for _,_,file in walk(dir_):
    for f in file:
        data = pd.read_table(dir_+'/'+f,names = names)
        Dec.append(data)
    print('Reading Finished. There are '+str(len(Dec))+' files in all.')

Reading Finished. There are 32 files in all.


In [7]:
Nov[1].shape

(4745086, 8)

In [8]:
timeseries = set(Nov[0].timeInterval)

In [18]:
t = sorted(list(timeseries))

In [28]:
tt = [Nov[0].timeInterval[0]+i*600000*gran for i in range(144//gran)]

In [32]:
sheet = Nov[0].fillna(0)

In [43]:
s = sheet.groupby(['timeInterval','squareId']).sum().drop(['countryCode'],axis = 1)

In [73]:
b = sheet[sheet.timeInterval == 1383260400000][sheet.squareId == 1].sum(axis = 0).drop(['countryCode'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [93]:
values = pd.Series([0,0,0,0,0,0,0],index = ['squareId','timeInterval','smsIn','smsOut','callIn','callOut','Internet'])

In [94]:
values

squareId        0
timeInterval    0
smsIn           0
smsOut          0
callIn          0
callOut         0
Internet        0
dtype: int64

In [95]:
b+b+values

squareId        4.000000e+00
timeInterval    5.533042e+12
smsIn           4.464538e-01
smsOut          3.135740e-01
callIn          3.218759e-01
callOut         1.045497e-01
Internet        2.205673e+01
dtype: float64

In [78]:
a

squareId        0.0
timeInterval    0.0
smsIn           0.0
smsOut          0.0
callIn          0.0
callOut         0.0
Internet        0.0
dtype: float64

In [54]:
type(s)

pandas.core.frame.DataFrame

In [61]:
gridNum = 2
for i in range(1,gridNum):
    gridData = pd.DataFrame()
    for sheet in Nov[:3]:
        tmp = sheet[sheet['squareId'] == i]
        gridData = pd.concat([gridData,tmp],axis = 0).reset_index(drop = True)
    print(gridData)

      squareId   timeInterval  countryCode     smsIn    smsOut    callIn  \
0            1  1383260400000            0  0.081363       NaN       NaN   
1            1  1383260400000           39  0.141864  0.156787  0.160938   
2            1  1383261000000            0  0.136588       NaN       NaN   
3            1  1383261000000           33       NaN       NaN       NaN   
4            1  1383261000000           39  0.278452  0.119926  0.188777   
5            1  1383261600000            0  0.053438       NaN       NaN   
6            1  1383261600000           39  0.330641  0.170952  0.134176   
7            1  1383262200000            0  0.026137       NaN       NaN   
8            1  1383262200000           39  0.681434  0.220815  0.027300   
9            1  1383262800000            0  0.027300       NaN       NaN   
10           1  1383262800000           39  0.243378  0.192891  0.053438   
11           1  1383263400000            0  0.027300       NaN       NaN   
12          

In [127]:
temporal_gran = 3
spatial_gran = 2
for sheet in Nov[:1]:
    #generate time interval
    timeInter = [sheet.timeInterval[0]+i*600000*temporal_gran for i in range(144//temporal_gran)]
    sheet = sheet.fillna(0)
    datamatrix = np.zeros(shape = (len(timeInter),1,100//spatial_gran,100//spatial_gran))
    for idz,t in enumerate(timeInter):
        for Row in range(0,100//spatial_gran):
            for Col in range(0, 100//spatial_gran):
                gridRow = 1 + Row * spatial_gran
                gridCol = 1 + Col * spatial_gran
                value = pd.Series([0,0,0,0,0,0,0],index = ['squareId','timeInterval','smsIn','smsOut','callIn','callOut','Internet']) 
                for g in range(temporal_gran):
                    for i in range(spatial_gran):                            
                        for j in range(spatial_gran):
                            print(g,i,j)
                            print(t + 600000*g,100*(gridRow+i-1)+gridCol+j)
                            value += sheet[sheet.timeInterval == t + 600000*g][sheet.squareId == 100*(gridRow+i-1)+gridCol+j].sum(axis = 0).drop(['countryCode'])
                    #print(value)
                datamatrix[idz,0,Row,Col]
        break
        

0 0 0
1383260400000 1
0 0 1
1383260400000 2
0 1 0
1383260400000 101
0 1 1
1383260400000 102
squareId        6.150000e+02
timeInterval    1.383260e+13
smsIn           8.908825e-01
smsOut          5.807865e-01
callIn          6.343068e-01
callOut         2.280560e-01
Internet        4.304581e+01
dtype: float64
1 0 0
1383261000000 1
1 0 1
1383261000000 2
1 1 0
1383261000000 101
1 1 1
1383261000000 102
squareId        1.335000e+03
timeInterval    3.181500e+13
smsIn           2.506234e+00
smsOut          1.057781e+00
callIn          1.378454e+00
callOut         8.958813e-01
Internet        8.662933e+01
dtype: float64
2 0 0
1383261600000 1
2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


 0 1
1383261600000 2
2 1 0
1383261600000 101
2 1 1
1383261600000 102
squareId        1.747000e+03
timeInterval    4.288109e+13
smsIn           3.994336e+00
smsOut          1.719618e+00
callIn          1.909379e+00
callOut         1.126584e+00
Internet        1.290828e+02
dtype: float64
0 0 0
1383260400000 3
0 0 1
1383260400000 4
0 1 0
1383260400000 103
0 1 1
1383260400000 104
squareId        4.280000e+02
timeInterval    1.106608e+13
smsIn           8.895253e-01
smsOut          5.969143e-01
callIn          6.569672e-01
callOut         2.171342e-01
Internet        4.421190e+01
dtype: float64
1 0 0
1383261000000 3
1 0 1
1383261000000 4
1 1 0
1383261000000 103
1 1 1
1383261000000 104
squareId        1.070000e+03
timeInterval    2.766522e+13
smsIn           2.553810e+00
smsOut          1.059492e+00
callIn          1.420111e+00
callOut         8.741014e-01
Internet        8.885490e+01
dtype: float64
2 0 0
1383261600000 3
2 0 1
1383261600000 4
2 1 0
1383261600000 103
2 1 1
1383261600000 104
s

dtype: float64
0 0 0
1383260400000 21
0 0 1
1383260400000 22
0 1 0
1383260400000 121
0 1 1
1383260400000 122
squareId        5.720000e+02
timeInterval    1.106608e+13
smsIn           7.083665e-01
smsOut          3.541833e-01
callIn          8.854581e-02
callOut         1.770916e-01
Internet        1.500670e+01
dtype: float64
1 0 0
1383261000000 21
1 0 1
1383261000000 22
1 1 0
1383261000000 121
1 1 1
1383261000000 122
squareId        1.430000e+03
timeInterval    2.766522e+13
smsIn           1.151096e+00
smsOut          7.083665e-01
callIn          1.770916e-01
callOut         5.062880e-01
Internet        2.731845e+01
dtype: float64
2 0 0
1383261600000 21
2 0 1
1383261600000 22
2 1 0
1383261600000 121
2 1 1
1383261600000 122
squareId        1.716000e+03
timeInterval    3.319826e+13
smsIn           1.543851e+00
smsOut          1.062550e+00
callIn          1.770916e-01
callOut         5.062880e-01
Internet        3.881970e+01
dtype: float64
0 0 0
1383260400000 23
0 0 1
1383260400000 24
0 1

0 1 1
1383260400000 140
squareId        6.780000e+02
timeInterval    9.682823e+12
smsIn           9.550924e-01
smsOut          4.024815e-01
callIn          8.794334e-03
callOut         2.723484e-01
Internet        2.820426e+01
dtype: float64
1 0 0
1383261000000 39
1 0 1
1383261000000 40
1 1 0
1383261000000 139
1 1 1
1383261000000 140
squareId        1.216000e+03
timeInterval    1.798239e+13
smsIn           1.368714e+00
smsOut          7.143615e-01
callIn          4.740479e-01
callOut         8.027941e-01
Internet        5.810862e+01
dtype: float64
2 0 0
1383261600000 39
2 0 1
1383261600000 40
2 1 0
1383261600000 139
2 1 1
1383261600000 140
squareId        2.212000e+03
timeInterval    3.181500e+13
smsIn           2.468734e+00
smsOut          1.506608e+00
callIn          8.578369e-01
callOut         1.592486e+00
Internet        8.366815e+01
dtype: float64
0 0 0
1383260400000 41
0 0 1
1383260400000 42
0 1 0
1383260400000 141
0 1 1
1383260400000 142
squareId        8.730000e+02
timeInterva

...

squareId        2.368680e+05
timeInterval    3.319826e+13
smsIn           7.779458e+00
smsOut          3.431363e+00
callIn          3.776930e+00
callOut         4.251042e+00
Internet        3.173917e+02
dtype: float64
2 0 0
1383261600000 9819
2 0 1
1383261600000 9820
2 1 0
1383261600000 9919
2 1 1
1383261600000 9920
squareId        3.158240e+05
timeInterval    4.426435e+13
smsIn           1.082920e+01
smsOut          6.374400e+00
callIn          4.044719e+00
callOut         4.712680e+00
Internet        4.466220e+02
dtype: float64
0 0 0
1383260400000 9821
0 0 1
1383260400000 9822
0 1 0
1383260400000 9921
0 1 1
1383260400000 9922
squareId        1.184580e+05
timeInterval    1.659912e+13
smsIn           4.516741e+00
smsOut          1.330424e+00
callIn          2.465477e+00
callOut         2.271628e+00
Internet        1.004213e+02
dtype: float64
1 0 0
1383261000000 9821
1 0 1
1383261000000 9822
1 1 0
1383261000000 9921
1 1 1
1383261000000 9922
squareId        2.369160e+05
timeInterval    3

squareId        8.903700e+04
timeInterval    1.244935e+13
smsIn           5.794112e-01
smsOut          1.001184e+00
callIn          1.567322e-01
callOut         5.258110e-02
Internet        1.127069e+02
dtype: float64
2 0 0
1383261600000 9837
2 0 1
1383261600000 9838
2 1 0
1383261600000 9937
2 1 1
1383261600000 9938
squareId        1.285870e+05
timeInterval    1.798239e+13
smsIn           1.138239e+00
smsOut          1.056962e+00
callIn          2.310187e-01
callOut         5.625983e-02
Internet        1.574521e+02
dtype: float64
0 0 0
1383260400000 9839
0 0 1
1383260400000 9840
0 1 0
1383260400000 9939
0 1 1
1383260400000 9940
squareId        3.955800e+04
timeInterval    5.533042e+12
smsIn           4.402150e-01
smsOut          0.000000e+00
callIn          3.561690e-01
callOut         0.000000e+00
Internet        6.816105e+01
dtype: float64
1 0 0
1383261000000 9839
1 0 1
1383261000000 9840
1 1 0
1383261000000 9939
1 1 1
1383261000000 9940
squareId        7.911600e+04
timeInterval    1

1 1 0
1383261000000 9955
1 1 1
1383261000000 9956
squareId        2.575460e+05
timeInterval    3.596478e+13
smsIn           1.147589e+01
smsOut          1.305335e+01
callIn          4.598368e+00
callOut         1.906640e+00
Internet        4.424915e+02
dtype: float64
2 0 0
1383261600000 9855
2 0 1
1383261600000 9856
2 1 0
1383261600000 9955
2 1 1
1383261600000 9956
squareId        3.169800e+05
timeInterval    4.426435e+13
smsIn           1.445129e+01
smsOut          1.735232e+01
callIn          6.815471e+00
callOut         3.705192e+00
Internet        6.574406e+02
dtype: float64
0 0 0
1383260400000 9857
0 0 1
1383260400000 9858
0 1 0
1383260400000 9957
0 1 1
1383260400000 9958
squareId        1.585200e+05
timeInterval    2.213217e+13
smsIn           4.872822e+00
smsOut          2.778560e+00
callIn          2.414848e+00
callOut         2.726725e+00
Internet        2.345856e+02
dtype: float64
1 0 0
1383261000000 9857
1 0 1
1383261000000 9858
1 1 0
1383261000000 9957
1 1 1
1383261000000 9

1 1 1
1383261000000 9974
squareId        2.379650e+05
timeInterval    3.319826e+13
smsIn           3.840942e+00
smsOut          2.048801e+00
callIn          2.743084e+00
callOut         1.832369e+00
Internet        1.982013e+02
dtype: float64
2 0 0
1383261600000 9873
2 0 1
1383261600000 9874
2 1 0
1383261600000 9973
2 1 1
1383261600000 9974
squareId        2.975070e+05
timeInterval    4.149783e+13
smsIn           5.496329e+00
smsOut          3.631256e+00
callIn          2.743084e+00
callOut         2.166940e+00
Internet        2.947605e+02
dtype: float64
0 0 0
1383260400000 9875
0 0 1
1383260400000 9876
0 1 0
1383260400000 9975
0 1 1
1383260400000 9976
squareId        1.191060e+05
timeInterval    1.659912e+13
smsIn           1.044822e+01
smsOut          4.463605e+00
callIn          2.502428e+00
callOut         3.381576e+00
Internet        2.978065e+02
dtype: float64
1 0 0
1383261000000 9875
1 0 1
1383261000000 9876
1 1 0
1383261000000 9975
1 1 1
1383261000000 9976
squareId        2.482

squareId        2.682710e+05
timeInterval    3.734804e+13
smsIn           8.354852e+00
smsOut          7.079972e+00
callIn          6.112327e+00
callOut         3.323546e+00
Internet        4.585193e+02
dtype: float64
2 0 0
1383261600000 9891
2 0 1
1383261600000 9892
2 1 0
1383261600000 9991
2 1 1
1383261600000 9992
squareId        3.774780e+05
timeInterval    5.256392e+13
smsIn           1.287646e+01
smsOut          1.133346e+01
callIn          8.537141e+00
callOut         6.121900e+00
Internet        6.680065e+02
dtype: float64
0 0 0
1383260400000 9893
0 0 1
1383260400000 9894
0 1 0
1383260400000 9993
0 1 1
1383260400000 9994
squareId        1.191210e+05
timeInterval    1.659912e+13
smsIn           3.089522e+00
smsOut          2.515106e+00
callIn          1.093265e+00
callOut         2.870653e-01
Internet        9.405592e+01
dtype: float64
1 0 0
1383261000000 9893
1 0 1
1383261000000 9894
1 1 0
1383261000000 9993
1 1 1
1383261000000 9994
squareId        2.085610e+05
timeInterval    2

In [7]:
import h5py

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [54]:
temporal_gran = 3
spatial_gran = 2
slots = 4320 // temporal_gran
rows  = 100 // spatial_gran
cols = 100 // spatial_gran
f = h5py.File('test.h5','w')
f_time = f.create_dataset('date',(slots,),dtype = 'S13')
f_data = f.create_dataset('data',(slots,1,rows,cols),dtype='float64')

In [45]:
ii = [i for i in range(1440)]
test = np.array(ii).astype('S10')

In [46]:
test

array([b'0', b'1', b'2', ..., b'1437', b'1438', b'1439'], dtype='|S10')

In [47]:
f_data[0,0,0,0] = 3
f_time = test
f.close()

In [48]:
ff = h5py.File('test.h5','r')

In [49]:
ddd = ff['date']

In [51]:
ddd[3]
ff.close()

In [58]:
f.close()

In [59]:
temporal_gran = 3
spatial_gran = 2
slots = 4320 // temporal_gran
rows  = 100 // spatial_gran
cols = 100 // spatial_gran
f = h5py.File('test3.h5','w')
f_time = f.create_dataset('date',(slots,),dtype = 'S10')
f_data = f.create_dataset('data',(slots,1,rows,cols),dtype='float64')
idzz = 0
for sheet in Nov:
    #generate time interval
    timeInter = [sheet.timeInterval[0]+i*600000*temporal_gran for i in range(144//temporal_gran)]
    sheet = sheet.fillna(0)
    #datamatrix = np.zeros(shape = (len(timeInter),1,100//spatial_gran,100//spatial_gran))
    for idz,t in enumerate(timeInter):
        for Row in range(0,100//spatial_gran):
            for Col in range(0, 100//spatial_gran):
                gridRow = 1 + Row * spatial_gran
                gridCol = 1 + Col * spatial_gran
                value = pd.Series([0,0,0,0,0,0,0],index = ['squareId','timeInterval','smsIn','smsOut','callIn','callOut','Internet']) 
                for g in range(temporal_gran):
                    for i in range(spatial_gran):                            
                        for j in range(spatial_gran):
                            print(g,i,j)
                            print(t + 600000*g,100*(gridRow+i-1)+gridCol+j)
                            value += sheet[sheet.timeInterval == t + 600000*g][sheet.squareId == 100*(gridRow+i-1)+gridCol+j].sum(axis = 0).drop(['countryCode'])
                    #print(value)
                #datamatrix[idz,0,Row,Col]
                f_data[idzz,0,Row,Col] = value['Internet']
        f_time[idzz] = bytes(str(t).encode(encoding='utf-8'))
        idzz += 1
f.close()
        

0 0 0
1383260400000 1
0 0 1
1383260400000 2
0 1 0
1383260400000 101
0 1 1
1383260400000 102
1 0 0
1383261000000 1
1 0 1
1383261000000 2
1 1 0
1383261000000 101
1 1 1
1383261000000 102
2 0 0
1383261600000 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


2 0 1
1383261600000 2
2 1 0
1383261600000 101
2 1 1
1383261600000 102
0 0 0
1383260400000 3
0 0 1
1383260400000 4
0 1 0
1383260400000 103
0 1 1
1383260400000 104
1 0 0
1383261000000 3
1 0 1
1383261000000 4
1 1 0
1383261000000 103
1 1 1
1383261000000 104
2 0 0
1383261600000 3
2 0 1
1383261600000 4
2 1 0
1383261600000 103
2 1 1
1383261600000 104
0 0 0
1383260400000 5
0 0 1
1383260400000 6
0 1 0
1383260400000 105
0 1 1
1383260400000 106
1 0 0
1383261000000 5
1 0 1
1383261000000 6
1 1 0
1383261000000 105
1 1 1
1383261000000 106
2 0 0
1383261600000 5
2 0 1
1383261600000 6
2 1 0
1383261600000 105
2 1 1
1383261600000 106
0 0 0
1383260400000 7
0 0 1
1383260400000 8
0 1 0
1383260400000 107
0 1 1
1383260400000 108
1 0 0
1383261000000 7
1 0 1
1383261000000 8
1 1 0
1383261000000 107
1 1 1
1383261000000 108
2 0 0
1383261600000 7
2 0 1
1383261600000 8
2 1 0
1383261600000 107
2 1 1
1383261600000 108
0 0 0
1383260400000 9
0 0 1
1383260400000 10
0 1 0
1383260400000 109
0 1 1
1383260400000 110
1 0 0
138

2 1 1
1383261600000 160
0 0 0
1383260400000 61
0 0 1
1383260400000 62
0 1 0
1383260400000 161
0 1 1
1383260400000 162
1 0 0
1383261000000 61
1 0 1
1383261000000 62
1 1 0
1383261000000 161
1 1 1
1383261000000 162
2 0 0
1383261600000 61
2 0 1
1383261600000 62
2 1 0
1383261600000 161
2 1 1
1383261600000 162
0 0 0
1383260400000 63
0 0 1
1383260400000 64
0 1 0
1383260400000 163
0 1 1
1383260400000 164
1 0 0
1383261000000 63
1 0 1
1383261000000 64
1 1 0
1383261000000 163
1 1 1
1383261000000 164
2 0 0
1383261600000 63
2 0 1
1383261600000 64
2 1 0
1383261600000 163
2 1 1
1383261600000 164
0 0 0
1383260400000 65
0 0 1
1383260400000 66
0 1 0
1383260400000 165
0 1 1
1383260400000 166
1 0 0
1383261000000 65
1 0 1
1383261000000 66
1 1 0
1383261000000 165
1 1 1
1383261000000 166
2 0 0
1383261600000 65
2 0 1
1383261600000 66
2 1 0
1383261600000 165
2 1 1
1383261600000 166
0 0 0
1383260400000 67
0 0 1
1383260400000 68
0 1 0
1383260400000 167
0 1 1
1383260400000 168
1 0 0
1383261000000 67
1 0 1
1383261

0 0 0
1383260400000 219
0 0 1
1383260400000 220
0 1 0
1383260400000 319
0 1 1
1383260400000 320
1 0 0
1383261000000 219
1 0 1
1383261000000 220
1 1 0
1383261000000 319
1 1 1
1383261000000 320
2 0 0
1383261600000 219
2 0 1
1383261600000 220
2 1 0
1383261600000 319
2 1 1
1383261600000 320
0 0 0
1383260400000 221
0 0 1
1383260400000 222
0 1 0
1383260400000 321
0 1 1
1383260400000 322
1 0 0
1383261000000 221
1 0 1
1383261000000 222
1 1 0
1383261000000 321
1 1 1
1383261000000 322
2 0 0
1383261600000 221
2 0 1
1383261600000 222
2 1 0
1383261600000 321
2 1 1
1383261600000 322
0 0 0
1383260400000 223
0 0 1
1383260400000 224
0 1 0
1383260400000 323
0 1 1
1383260400000 324
1 0 0
1383261000000 223
1 0 1
1383261000000 224
1 1 0
1383261000000 323
1 1 1
1383261000000 324
2 0 0
1383261600000 223
2 0 1
1383261600000 224
2 1 0
1383261600000 323
2 1 1
1383261600000 324
0 0 0
1383260400000 225
0 0 1
1383260400000 226
0 1 0
1383260400000 325
0 1 1
1383260400000 326
1 0 0
1383261000000 225
1 0 1
1383261000

2 1 0
1383261600000 375
2 1 1
1383261600000 376
0 0 0
1383260400000 277
0 0 1
1383260400000 278
0 1 0
1383260400000 377
0 1 1
1383260400000 378
1 0 0
1383261000000 277
1 0 1
1383261000000 278
1 1 0
1383261000000 377
1 1 1
1383261000000 378
2 0 0
1383261600000 277
2 0 1
1383261600000 278
2 1 0
1383261600000 377
2 1 1
1383261600000 378
0 0 0
1383260400000 279
0 0 1
1383260400000 280
0 1 0
1383260400000 379
0 1 1
1383260400000 380
1 0 0
1383261000000 279
1 0 1
1383261000000 280
1 1 0
1383261000000 379
1 1 1
1383261000000 380
2 0 0
1383261600000 279
2 0 1
1383261600000 280
2 1 0
1383261600000 379
2 1 1
1383261600000 380
0 0 0
1383260400000 281
0 0 1
1383260400000 282
0 1 0
1383260400000 381
0 1 1
1383260400000 382
1 0 0
1383261000000 281
1 0 1
1383261000000 282
1 1 0
1383261000000 381
1 1 1
1383261000000 382
2 0 0
1383261600000 281
2 0 1
1383261600000 282
2 1 0
1383261600000 381
2 1 1
1383261600000 382
0 0 0
1383260400000 283
0 0 1
1383260400000 284
0 1 0
1383260400000 383
0 1 1
1383260400

1 0 1
1383261000000 434
1 1 0
1383261000000 533
1 1 1
1383261000000 534
2 0 0
1383261600000 433
2 0 1
1383261600000 434
2 1 0
1383261600000 533
2 1 1
1383261600000 534
0 0 0
1383260400000 435
0 0 1
1383260400000 436
0 1 0
1383260400000 535
0 1 1
1383260400000 536
1 0 0
1383261000000 435
1 0 1
1383261000000 436
1 1 0
1383261000000 535
1 1 1
1383261000000 536
2 0 0
1383261600000 435
2 0 1
1383261600000 436
2 1 0
1383261600000 535
2 1 1
1383261600000 536
0 0 0
1383260400000 437
0 0 1
1383260400000 438
0 1 0
1383260400000 537
0 1 1
1383260400000 538
1 0 0
1383261000000 437
1 0 1
1383261000000 438
1 1 0
1383261000000 537
1 1 1
1383261000000 538
2 0 0
1383261600000 437
2 0 1
1383261600000 438
2 1 0
1383261600000 537
2 1 1
1383261600000 538
0 0 0
1383260400000 439
0 0 1
1383260400000 440
0 1 0
1383260400000 539
0 1 1
1383260400000 540
1 0 0
1383261000000 439
1 0 1
1383261000000 440
1 1 0
1383261000000 539
1 1 1
1383261000000 540
2 0 0
1383261600000 439
2 0 1
1383261600000 440
2 1 0
1383261600

KeyboardInterrupt: 

In [122]:
tmp = np.ndarray(shape = (10,2,2,2))

In [123]:
tmp[1,1,1,1] = 3

In [124]:
tmp

array([[[[1.15438005e-311, 0.00000000e+000],
         [8.27578358e-313, 1.15436571e-311]],

        [[1.15438008e-311, 5.92878775e-323],
         [1.15523281e-311, 1.03977794e-312]]],


       [[[4.47593775e-091, 0.00000000e+000],
         [4.47593816e-091, 0.00000000e+000]],

        [[1.06099790e-312, 4.47593775e-091],
         [1.15438008e-311, 3.00000000e+000]]],


       [[[1.15523281e-311, 1.35807731e-312],
         [4.91018063e+252, 0.00000000e+000]],

        [[1.31345109e-047, 0.00000000e+000],
         [1.37929726e-312, 4.56335146e-072]]],


       [[[1.15438008e-311, 1.17075663e+214],
         [1.15523281e-311, 1.52783697e-312]],

        [[1.26931819e-076, 0.00000000e+000],
         [6.01347002e-154, 0.00000000e+000]]],


       [[[1.54905693e-312, 1.96872154e-153],
         [1.15438008e-311, 3.45365695e+175]],

        [[1.15523281e-311, 1.71881659e-312],
         [6.97843598e+252, 0.00000000e+000]]],


       [[[7.25154833e+271, 0.00000000e+000],
         [1.74003655e-312

In [63]:
1383260400000+141*600000

1383345000000

In [61]:
v = pd.Series([0,0,0,0,0,0,0],index = ['squareId','timeInterval','smsIn','smsOut','callIn','callOut','Internet']) 
for k in [1383260400000,1383261000000,1383261600000]:
    for i in [1,2,101,102]:
        v += sheet[(sheet.timeInterval == k)&(sheet.squareId == i)].sum(axis = 0).drop(['countryCode'])
    print(v)
v    

squareId        6.150000e+02
timeInterval    1.383260e+13
smsIn           8.908825e-01
smsOut          5.807865e-01
callIn          6.343068e-01
callOut         2.280560e-01
Internet        4.304581e+01
dtype: float64
squareId        1.335000e+03
timeInterval    3.181500e+13
smsIn           2.506234e+00
smsOut          1.057781e+00
callIn          1.378454e+00
callOut         8.958813e-01
Internet        8.662933e+01
dtype: float64
squareId        1.747000e+03
timeInterval    4.288109e+13
smsIn           3.994336e+00
smsOut          1.719618e+00
callIn          1.909379e+00
callOut         1.126584e+00
Internet        1.290828e+02
dtype: float64


squareId        1.747000e+03
timeInterval    4.288109e+13
smsIn           3.994336e+00
smsOut          1.719618e+00
callIn          1.909379e+00
callOut         1.126584e+00
Internet        1.290828e+02
dtype: float64

In [98]:
s

smsIn    smsOut    callIn   callOut    Internet
timeInterval  squareId                                                    
1383260400000 1         0.223227  0.156787  0.160938  0.052275   11.028366
              2         0.222201  0.147617  0.164946  0.054712   11.058225
              3         0.221109  0.137855  0.169213  0.057306   11.090008
              4         0.226198  0.183349  0.149327  0.045216   10.941881
              5         0.205065  0.175393  0.139139  0.043455    9.916548
              6         0.221109  0.137855  0.169213  0.057306   11.090008
              7         0.221109  0.137855  0.169213  0.057306   11.090008
              8         0.221109  0.137855  0.169213  0.057306   11.090008
              9         0.221109  0.137855  0.169213  0.057306   11.090008
              10        0.118350  0.069349  0.134960  0.057306    6.374815
              11        0.063919  0.018105  0.081655  0.045446    3.229855
              12        0.128074  0.026518  0.027959  0.027959    2.634588
              13        0.060948  0.003447  0.019711  0.018679    1.526482
              14        0.075953  0.000000  0.024294  0.021224    2.530250
              15        0.075269  0.000000  0.025090  0.025090    2.639616
              16        0.128775  0.046529  0.023538  0.035170    3.195412
              17        0.177092  0.088546  0.022136  0.044273    3.751674
              18        0.177092  0.088546  0.022136  0.044273    3.751674
              19        0.177092  0.088546  0.022136  0.044273    3.751674
              20        0.177092  0.088546  0.022136  0.044273    3.751674
              21        0.177092  0.088546  0.022136  0.044273    3.751674
              22        0.177092  0.088546  0.022136  0.044273    3.751674
              23        0.185353  0.096807  0.022136  0.044273    3.846671
              24        0.245099  0.156553  0.022136  0.044273    4.533661
              25        0.246803  0.158257  0.022136  0.044273    4.553261
              26        0.246803  0.158257  0.022136  0.044273    4.553261
              27        0.211338  0.147966  0.019564  0.044088    4.667853
              28        0.153113  0.123007  0.019415  0.043816    5.124863
              29        0.113637  0.082296  0.031341  0.043729    6.228075
              30        0.101041  0.062408  0.038633  0.043729    6.810123
...                          ...       ...       ...       ...         ...
1383346200000 9971      1.251692  0.313917  0.150528  0.180840   21.311994
              9972      0.990440  0.000000  0.000000  0.038194   11.890972
              9973      0.626165  0.000000  0.000000  0.190681   16.592663
              9974      0.423612  0.000000  0.000000  0.404731   19.991220
              9975      0.625941  0.151387  0.218242  0.726061   31.012615
              9976      2.981209  0.933876  0.615569  0.818445  139.655342
              9977      3.335812  1.058924  0.499265  0.779094  155.334799
              9978      3.257095  1.043181  0.483521  0.763351  152.545068
              9979      1.287929  0.654373  0.089700  0.364456   83.910336
              9980      0.836227  0.755124  0.000000  0.080256   59.268162
              9981      0.650434  0.453530  0.000000  0.019953   45.952203
              9982      1.939388  1.750466  0.000000  0.042596   43.664536
              9983      3.225673  2.933427  0.000000  0.000000   37.273459
              9984      1.132582  1.507185  0.076005  0.116613   99.487309
              9985      1.667723  2.368971  0.563107  0.236447   95.009262
              9986      0.698188  2.679221  1.120336  0.012614   67.970446
              9987      0.563918  2.302459  1.083291  0.004656   63.273789
              9988      0.797725  0.318085  0.516070  0.276422   33.415269
              9989      1.142628  0.148634  0.610972  0.798492   50.541585
              9990      0.859693  0.319653  0.598340  0.673943   55.085738
              9991      0.560141  0.477973  0.418263

In [65]:
dir_ = 'D:/MobileTrafficPrediction/data/interim/grid5050'
chunksize = 100
#for i in range(1,2):
data = pd.read_csv('{}/grid{}.csv'.format(dir_,1))
data['time'] = data['time'].apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))*1000))

In [151]:
data

,time,sms_in,sms_out,call_in,call_out,traffic
0,1383260400000,7.821092,7.146914,2.488899,0.975139,256.746069
1,1383262200000,18.022091,8.551941,1.258964,1.680561,236.098269
2,1383264000000,3.821885,4.610790,1.725282,0.968014,231.317465
3,1383265800000,4.333531,5.480197,0.799714,1.316552,197.893439
4,1383267600000,1.826368,4.064264,0.409973,1.224156,168.592148
5,1383269400000,2.513358,6.035991,0.235961,0.166507,175.371374
6,1383271200000,1.503102,1.911641,0.117576,0.320128,155.576252
7,1383273000000,1.167845,3.101185,0.111005,0.000000,142.571439
8,1383274800000,1.502464,2.413542,0.055502,0.055502,162.721161
9,1383276600000,1.287164,2.392810,0.403402,0.292397,182.198179


In [71]:
print(data.iloc[0][0])

2013-11-01 07:00:00


In [72]:
import time
interval = time.strptime(data.iloc[0][0], "%Y-%m-%d %H:%M:%S")

In [137]:
dir_ = 'D:/MobileTrafficPrediction/data/interim/grid5050'
temporal_gran = 3
spatial_gran = 2
slots = 4320 // temporal_gran
rows  = 100 // spatial_gran
cols = 100 // spatial_gran
f = h5py.File('Nov_internet_data.h5','w')
f_time = f.create_dataset('date',(slots,),dtype = 'S13')
f_data = f.create_dataset('data',(slots,1,rows,cols),dtype='float64')
start_file = 1
#generate time interval
timeInter = [1383260400000+i*600000*temporal_gran for i in range(1440)]
for i,t in enumerate(timeInter):
    f_time[i] = bytes(str(t).encode(encoding='utf-8'))
    
for grid in range(1,2501):
    data = pd.read_csv('{}/grid{}.csv'.format(dir_,grid))
    data['time'] = data['time'].apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))*1000))
    Row = math.ceil(grid/50)
    Row -= 1
    Col = 50 if grid % 50 == 0 else grid % 50
    Col -= 1
    for i,t in enumerate(timeInter):
        f_data[i,0,Row,Col] = float(data[(data['time'] == t)].traffic)
        if i % 1440 == 100:
            print(grid,i,0,Row,Col,f_data[i,0,Row,Col])
f.close()

1 100 0 0 0 78.85178893990438
2 100 0 0 1 81.31284537963478
3 100 0 0 2 79.46619298448836
4 100 0 0 3 81.76802588652735
5 100 0 0 4 64.9811228495269
6 100 0 0 5 18.059436259601053
7 100 0 0 6 14.250624553190448
8 100 0 0 7 22.95703401910484
9 100 0 0 8 30.349138394287706
10 100 0 0 9 30.349138394287706
11 100 0 0 10 30.349138394287706
12 100 0 0 11 43.713120066274406
13 100 0 0 12 55.98761876319043
14 100 0 0 13 60.910637581524504
15 100 0 0 14 61.384899515034476
16 100 0 0 15 68.46903256739049
17 100 0 0 16 75.66709981769431
18 100 0 0 17 57.9244685952295
19 100 0 0 18 58.21454596519122
20 100 0 0 19 61.954662545114346
21 100 0 0 20 69.851384450252
22 100 0 0 21 53.312091421972674
23 100 0 0 22 159.2891869589437
24 100 0 0 23 1110.294711185052
25 100 0 0 24 245.10512173789985
26 100 0 0 25 138.84911393348963
27 100 0 0 26 133.0978386882154
28 100 0 0 27 162.12268755902736
29 100 0 0 28 175.05287019702354
30 100 0 0 29 165.76030108641137
31 100 0 0 30 53.13609005573996
32 100 0 0 31 56

252 100 0 5 1 38.58710114613831
253 100 0 5 2 26.3648116787787
254 100 0 5 3 25.68296268843541
255 100 0 5 4 31.101169158236367
256 100 0 5 5 32.25590420950745
257 100 0 5 6 36.18291671051879
258 100 0 5 7 39.35673913259421
259 100 0 5 8 32.590817211071034
260 100 0 5 9 30.753137208177968
261 100 0 5 10 33.26777872678536
262 100 0 5 11 65.63800182568681
263 100 0 5 12 26.518352155868648
264 100 0 5 13 29.316987148388453
265 100 0 5 14 33.235924710818736
266 100 0 5 15 84.11959630791472
267 100 0 5 16 143.28522148882558
268 100 0 5 17 209.52143883309967
269 100 0 5 18 211.80913452856942
270 100 0 5 19 179.92523422800957
271 100 0 5 20 138.82486079542508
272 100 0 5 21 117.66608038513728
273 100 0 5 22 218.69452121090015
274 100 0 5 23 410.72570143816864
275 100 0 5 24 376.9734231048439
276 100 0 5 25 662.5781712322472
277 100 0 5 26 484.87289873681266
278 100 0 5 27 428.3666773169008
279 100 0 5 28 341.4826924787086
280 100 0 5 29 178.37167952326075
281 100 0 5 30 113.08428620834108
282

499 100 0 9 48 147.68078488098388
500 100 0 9 49 20.32074543530016
501 100 0 10 0 20.730785635513975
502 100 0 10 1 26.507904148369132
503 100 0 10 2 48.27408728147091
504 100 0 10 3 88.85140750233782
505 100 0 10 4 74.1006813175853
506 100 0 10 5 74.24753037857519
507 100 0 10 6 86.71503759337853
508 100 0 10 7 111.86015173045841
509 100 0 10 8 85.30376191050806
510 100 0 10 9 56.32699223468605
511 100 0 10 10 56.10656883303407
512 100 0 10 11 15.624265831328948
513 100 0 10 12 21.741904236259643
514 100 0 10 13 27.10158066087159
515 100 0 10 14 19.173667322412953
516 100 0 10 15 20.700706716110954
517 100 0 10 16 82.98076028848496
518 100 0 10 17 187.06690084981793
519 100 0 10 18 265.5150176738736
520 100 0 10 19 253.53497183343143
521 100 0 10 20 387.97559565951866
522 100 0 10 21 329.34066087014725
523 100 0 10 22 444.39872394378364
524 100 0 10 23 200.9899346386153
525 100 0 10 24 228.54135894035787
526 100 0 10 25 383.074666083794
527 100 0 10 26 356.81990185527235
528 100 0 10 

739 100 0 14 38 121.89021440069037
740 100 0 14 39 86.39086839353624
741 100 0 14 40 210.2853256361446
742 100 0 14 41 248.33884156875092
743 100 0 14 42 356.3512443408407
744 100 0 14 43 331.37486545776983
745 100 0 14 44 237.43885028332264
746 100 0 14 45 130.38311935130292
747 100 0 14 46 146.8038589160819
748 100 0 14 47 196.07909590651025
749 100 0 14 48 261.0327026012086
750 100 0 14 49 225.4198979934432
751 100 0 15 0 17.939810127815978
752 100 0 15 1 19.060380991450923
753 100 0 15 2 46.742748920812474
754 100 0 15 3 69.09700666636203
755 100 0 15 4 61.91609147419585
756 100 0 15 5 61.81168505090589
757 100 0 15 6 91.71799970830546
758 100 0 15 7 316.488821443677
759 100 0 15 8 799.9191804344587
760 100 0 15 9 541.9831961600928
761 100 0 15 10 525.3525198896409
762 100 0 15 11 142.8908878665385
763 100 0 15 12 62.99598597081502
764 100 0 15 13 227.73848185714692
765 100 0 15 14 264.2712482890842
766 100 0 15 15 291.14804332540183
767 100 0 15 16 582.9298035008359
768 100 0 15 1

979 100 0 19 28 2119.897862165345
980 100 0 19 29 1977.524463049846
981 100 0 19 30 1948.636264159658
982 100 0 19 31 1712.931876544828
983 100 0 19 32 1219.6152598214005
984 100 0 19 33 1285.8703705139212
985 100 0 19 34 1652.9105936636481
986 100 0 19 35 1681.9833898515149
987 100 0 19 36 910.2251866489868
988 100 0 19 37 539.1524256729717
989 100 0 19 38 383.5376380506775
990 100 0 19 39 381.80390434214166
991 100 0 19 40 387.084612712338
992 100 0 19 41 256.6706114004638
993 100 0 19 42 206.14114829494332
994 100 0 19 43 102.20411334282946
995 100 0 19 44 137.67861034475862
996 100 0 19 45 122.82767874282654
997 100 0 19 46 125.14749475934319
998 100 0 19 47 145.93234760379698
999 100 0 19 48 148.6839126809932
1000 100 0 19 49 146.22257855102603
1001 100 0 20 0 18.736577273762506
1002 100 0 20 1 58.17349775121646
1003 100 0 20 2 71.59434338802774
1004 100 0 20 3 69.83809501022739
1005 100 0 20 4 70.98072335455069
1006 100 0 20 5 83.91436086464516
1007 100 0 20 6 84.45440047165917
1

...

1446 100 0 28 45 161.87799376567327
1447 100 0 28 46 137.42347498153313
1448 100 0 28 47 108.56640197149132
1449 100 0 28 48 268.5387984536287
1450 100 0 28 49 367.32148135731916
1451 100 0 29 0 218.3315828734968
1452 100 0 29 1 98.41803295155928
1453 100 0 29 2 33.526724240700425
1454 100 0 29 3 22.072640582305592
1455 100 0 29 4 49.606614997944014
1456 100 0 29 5 110.63850286195446
1457 100 0 29 6 213.07672277766628
1458 100 0 29 7 497.4577160760666
1459 100 0 29 8 450.08833999960854
1460 100 0 29 9 63.31927516416065
1461 100 0 29 10 42.28645206721457
1462 100 0 29 11 51.556341472713356
1463 100 0 29 12 84.99506586317139
1464 100 0 29 13 97.57712200187487
1465 100 0 29 14 77.1159672833111
1466 100 0 29 15 38.82530807882439
1467 100 0 29 16 44.457894481872714
1468 100 0 29 17 126.55942852018255
1469 100 0 29 18 80.40644426506117
1470 100 0 29 19 102.18512321848543
1471 100 0 29 20 137.53701494537196
1472 100 0 29 21 629.4607971345514
1473 100 0 29 22 1120.0252764862787
1474 100 0 29 2

1679 100 0 33 28 2311.4034982953367
1680 100 0 33 29 1898.8795174992426
1681 100 0 33 30 1948.528696913504
1682 100 0 33 31 1551.8333326032634
1683 100 0 33 32 1164.7108262138795
1684 100 0 33 33 1294.1196874994025
1685 100 0 33 34 1502.176869180543
1686 100 0 33 35 3099.651690767125
1687 100 0 33 36 2043.4099390098272
1688 100 0 33 37 1428.3352793372437
1689 100 0 33 38 894.1812309027414
1690 100 0 33 39 751.1651578831378
1691 100 0 33 40 191.98863937017651
1692 100 0 33 41 422.2324271605194
1693 100 0 33 42 637.7149788586969
1694 100 0 33 43 575.732217616886
1695 100 0 33 44 378.7590471405073
1696 100 0 33 45 331.4258336366925
1697 100 0 33 46 602.664855206493
1698 100 0 33 47 799.9257174457225
1699 100 0 33 48 404.9274711508637
1700 100 0 33 49 292.3993660202421
1701 100 0 34 0 61.99222485545216
1702 100 0 34 1 87.71537440693761
1703 100 0 34 2 251.70528841629292
1704 100 0 34 3 261.0933215559969
1705 100 0 34 4 155.19557974528638
1706 100 0 34 5 165.4023435437943
1707 100 0 34 6 94

1912 100 0 38 11 64.47716033699685
1913 100 0 38 12 30.993066852698153
1914 100 0 38 13 26.390553215625385
1915 100 0 38 14 130.15806501556648
1916 100 0 38 15 149.06866634465868
1917 100 0 38 16 200.42717794893662
1918 100 0 38 17 294.5910916382024
1919 100 0 38 18 402.0894464451473
1920 100 0 38 19 684.3990448876939
1921 100 0 38 20 711.7910001691905
1922 100 0 38 21 994.8539404326871
1923 100 0 38 22 441.8300408009677
1924 100 0 38 23 252.1789308943007
1925 100 0 38 24 356.0059796538675
1926 100 0 38 25 843.6617870999864
1927 100 0 38 26 1022.3852978739819
1928 100 0 38 27 1184.6155016173545
1929 100 0 38 28 752.9575626442329
1930 100 0 38 29 464.33769463562254
1931 100 0 38 30 822.6894480589041
1932 100 0 38 31 745.0056533873195
1933 100 0 38 32 739.2642127184426
1934 100 0 38 33 977.1033818001019
1935 100 0 38 34 1464.6549692095334
1936 100 0 38 35 623.0109878941844
1937 100 0 38 36 443.54318551183815
1938 100 0 38 37 357.1009382454158
1939 100 0 38 38 418.4341541967965
1940 100 0

2145 100 0 42 44 479.9556854209711
2146 100 0 42 45 467.4119506110358
2147 100 0 42 46 312.2794126925969
2148 100 0 42 47 249.50401084234775
2149 100 0 42 48 187.71787352055316
2150 100 0 42 49 122.7659203361207
2151 100 0 43 0 317.02528755448355
2152 100 0 43 1 274.3032418790142
2153 100 0 43 2 286.4055358912252
2154 100 0 43 3 223.33690390637642
2155 100 0 43 4 292.0412802895005
2156 100 0 43 5 438.8879150580955
2157 100 0 43 6 339.40348861502997
2158 100 0 43 7 307.64583879587155
2159 100 0 43 8 493.9620864228802
2160 100 0 43 9 865.8267406634317
2161 100 0 43 10 719.7593088237504
2162 100 0 43 11 250.29424527233795
2163 100 0 43 12 255.60025052315635
2164 100 0 43 13 167.28212853164666
2165 100 0 43 14 215.47931165081658
2166 100 0 43 15 289.24245177800617
2167 100 0 43 16 462.8894969133683
2168 100 0 43 17 610.3678234287522
2169 100 0 43 18 523.5708019945025
2170 100 0 43 19 617.4119651049241
2171 100 0 43 20 302.5916295127588
2172 100 0 43 21 244.2992868132421
2173 100 0 43 22 21

2379 100 0 47 28 664.2359107280522
2380 100 0 47 29 832.2795568528554
2381 100 0 47 30 910.1668740771092
2382 100 0 47 31 379.97985768631827
2383 100 0 47 32 441.08574915560797
2384 100 0 47 33 1137.5561906890339
2385 100 0 47 34 790.4631326120963
2386 100 0 47 35 361.9129206562459
2387 100 0 47 36 372.690631256218
2388 100 0 47 37 621.2685067815263
2389 100 0 47 38 233.02939644065154
2390 100 0 47 39 89.56367642147586
2391 100 0 47 40 494.06094003919793
2392 100 0 47 41 872.1161320506857
2393 100 0 47 42 500.66714061333687
2394 100 0 47 43 325.2913014968646
2395 100 0 47 44 285.4943820176485
2396 100 0 47 45 314.6701422360887
2397 100 0 47 46 331.44935899310394
2398 100 0 47 47 419.8930948264466
2399 100 0 47 48 688.4086419761353
2400 100 0 47 49 563.6543996267883
2401 100 0 48 0 53.89151574001709
2402 100 0 48 1 45.04159731919764
2403 100 0 48 2 29.085762241885963
2404 100 0 48 3 17.657585560889302
2405 100 0 48 4 14.891276502238533
2406 100 0 48 5 14.74632464744256
2407 100 0 48 6 5

In [155]:
dir_ = 'D:/MobileTrafficPrediction/data/interim/grid5050'
temporal_gran = 3
spatial_gran = 2
slots = 4320 // temporal_gran
rows  = 100 // spatial_gran
cols = 100 // spatial_gran
f1 = h5py.File('Nov_taffic_data.h5','w')
f2 = h5py.File('Nov_sms_data.h5','w')
f3 = h5py.File('Nov_call_data.h5','w')

f1_time = f1.create_dataset('date',(slots,),dtype = 'S13')
f2_time = f2.create_dataset('date',(slots,),dtype = 'S13')
f3_time = f3.create_dataset('date',(slots,),dtype = 'S13')

f1_data = f1.create_dataset('data',(slots,1,rows,cols),dtype='float64')
f2_data = f2.create_dataset('data',(slots,2,rows,cols),dtype='float64')
f3_data = f3.create_dataset('data',(slots,2,rows,cols),dtype='float64')

start_file = 1
#generate time interval
timeInter = [1383260400000+i*600000*temporal_gran for i in range(1440)]
for i,t in enumerate(timeInter):
    f1_time[i] = bytes(str(t).encode(encoding='utf-8'))
    f2_time[i] = bytes(str(t).encode(encoding='utf-8'))
    f3_time[i] = bytes(str(t).encode(encoding='utf-8'))
    
for grid in range(1,2501):
    data = pd.read_csv('{}/grid{}.csv'.format(dir_,grid))
    data['time'] = data['time'].apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))*1000))
    Row = math.ceil(grid/50)
    Row -= 1
    Col = 50 if grid % 50 == 0 else grid % 50
    Col -= 1
    for i,t in enumerate(timeInter):
        f1_data[i,0,Row,Col] = float(data[(data['time'] == t)].traffic)
        f2_data[i,0,Row,Col] = float(data[(data['time'] == t)].sms_in)
        f2_data[i,1,Row,Col] = float(data[(data['time'] == t)].sms_out)
        f3_data[i,0,Row,Col] = float(data[(data['time'] == t)].call_in)
        f3_data[i,1,Row,Col] = float(data[(data['time'] == t)].call_out)
    print(grid,i,0,Row,Col)
f1.close()
f2.close()
f3.close()

1 1439 0 0 0
2 1439 0 0 1
3 1439 0 0 2
4 1439 0 0 3
5 1439 0 0 4
6 1439 0 0 5
7 1439 0 0 6
8 1439 0 0 7
9 1439 0 0 8
10 1439 0 0 9
11 1439 0 0 10
12 1439 0 0 11
13 1439 0 0 12
14 1439 0 0 13
15 1439 0 0 14
16 1439 0 0 15
17 1439 0 0 16
18 1439 0 0 17
19 1439 0 0 18
20 1439 0 0 19
21 1439 0 0 20
22 1439 0 0 21
23 1439 0 0 22
24 1439 0 0 23
25 1439 0 0 24
26 1439 0 0 25
27 1439 0 0 26
28 1439 0 0 27
29 1439 0 0 28
30 1439 0 0 29
31 1439 0 0 30
32 1439 0 0 31
33 1439 0 0 32
34 1439 0 0 33
35 1439 0 0 34
36 1439 0 0 35
37 1439 0 0 36
38 1439 0 0 37
39 1439 0 0 38
40 1439 0 0 39
41 1439 0 0 40
42 1439 0 0 41
43 1439 0 0 42
44 1439 0 0 43
45 1439 0 0 44
46 1439 0 0 45
47 1439 0 0 46
48 1439 0 0 47
49 1439 0 0 48
50 1439 0 0 49
51 1439 0 1 0
52 1439 0 1 1
53 1439 0 1 2
54 1439 0 1 3
55 1439 0 1 4
56 1439 0 1 5
57 1439 0 1 6
58 1439 0 1 7
59 1439 0 1 8
60 1439 0 1 9
61 1439 0 1 10
62 1439 0 1 11
63 1439 0 1 12
64 1439 0 1 13
65 1439 0 1 14
66 1439 0 1 15
67 1439 0 1 16
68 1439 0 1 17
69 1439 0

KeyboardInterrupt: 

In [152]:
data = pd.read_csv('{}/grid{}.csv'.format('D:/MobileTrafficPrediction/data/interim/grid5050',1))

In [154]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.data